<a href="https://colab.research.google.com/github/Gajendra-theDataEngineer004627/BigDataAssignments/blob/Assignment11/Assignment12.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!apt-get update -y

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [515 kB]
Get:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Get:10 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [1,283 kB]
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,260 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/restricted amd64 Packages [1,158 kB]
Get

In [3]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [4]:
!wget -q https://archive.apache.org/dist/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz

In [5]:
!tar xf spark-3.1.2-bin-hadoop2.7.tgz

In [6]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop2.7"

In [7]:
!pip install -q findspark
import findspark
findspark.init()

In [8]:
from pyspark.sql import SparkSession
spark = (SparkSession
 .builder
 .appName("<app_name>")
 .getOrCreate())

In [ ]:
# function to doanload data files from the google drive
import gdown
def downloadfiles_fromFolder() :
    url = "https://drive.google.com/drive/folders/1Hf8PijpBSNyDjwr-YgozGDRL8UqMDq3D?usp=sharing"   # can also be used input()
    if url.split("/")[-1]== "?usp=sharing":
       url = url.replace("?usp=sharing","")
    gdown.download_folder(url, output="/content")

downloadfiles_fromFolder()

In [10]:
# loading the data in dataframe and then validate accordingly

deptDf = spark.read.format("json").option("path","/content/dept.json").load()
empDf = spark.read.format("json").option("path","/content/employee.json").load()

# renamed the ambiguous columns for future usage
deptDf1 = deptDf.withColumnRenamed("deptid","deptid_dept")
empDf1 = empDf.withColumnRenamed("deptid","deptid_emp")
deptDf1.show()
empDf1.show(truncate =False)


+---------+-----------+
| deptName|deptid_dept|
+---------+-----------+
|       IT|         11|
|       HR|         21|
|Marketing|         31|
|      Fin|         41|
|    Admin|         51|
+---------+-----------+

+---------------------+---+----------+-------+----+------+
|address              |age|deptid_emp|empname|id  |salary|
+---------------------+---+----------+-------+----+------+
|[{Pune, Maharashtra}]|25 |11        |satish |1201|5000  |
|[{Pune, Maharashtra}]|28 |21        |krishna|1202|6000  |
|[{Pune, Maharashtra}]|39 |31        |amith  |1203|7000  |
|[{Pune, Maharashtra}]|23 |41        |javed  |1204|8000  |
|[{Pune, Maharashtra}]|23 |41        |prudvi |1205|9000  |
+---------------------+---+----------+-------+----+------+



In [11]:
"""
Problem 1: Given 2 Datasets employee.json and dept.json
We need to calculate the count of employees against each department. Use
Structured API’s.   header : depName,deptid,empcount"""
from pyspark.sql import SparkSession
from pyspark.sql.window import Window
from pyspark.sql import functions as F

join_condn = deptDf1.deptid_dept == empDf1.deptid_emp
joinedDf = deptDf1.join(empDf1,join_condn,"LEFT")
joinedDf.show(truncate = False)

result = joinedDf.select(
    F.col("deptName").alias("deptName"),
    F.col("deptid_dept").alias("deptid"),
    F.count("id").over(Window.partitionBy("deptid_dept").orderBy("deptid_dept")).alias("empCount")
).distinct().orderBy("deptid_dept")


result1 = joinedDf.selectExpr("deptName","deptid_dept as deptid",
                              "count(id)over(partition by deptid_dept order by deptid_dept) as empCount1").distinct().orderBy("deptid_dept")

# worked on both ways column object and using Sql Expression  final results
result.show()
result1.show()


+---------+-----------+---------------------+----+----------+-------+----+------+
|deptName |deptid_dept|address              |age |deptid_emp|empname|id  |salary|
+---------+-----------+---------------------+----+----------+-------+----+------+
|IT       |11         |[{Pune, Maharashtra}]|25  |11        |satish |1201|5000  |
|HR       |21         |[{Pune, Maharashtra}]|28  |21        |krishna|1202|6000  |
|Marketing|31         |[{Pune, Maharashtra}]|39  |31        |amith  |1203|7000  |
|Fin      |41         |[{Pune, Maharashtra}]|23  |41        |prudvi |1205|9000  |
|Fin      |41         |[{Pune, Maharashtra}]|23  |41        |javed  |1204|8000  |
|Admin    |51         |null                 |null|null      |null   |null|null  |
+---------+-----------+---------------------+----+----------+-------+----+------+

+---------+------+--------+
| deptName|deptid|empCount|
+---------+------+--------+
|       IT|    11|       1|
|       HR|    21|       1|
|Marketing|    31|       1|
|      Fin|

In [26]:
"""
Problem 3:
File A is a text file of size 1.2 GB in HDFS at location /loc/x. It contains match by
match statistics of runs scored by all the batsman in the history of cricket.
File B is a text file of size 1.2 MB present in local dir /loc/y. It contains list of
batsman playing in cricket world cup 2019.

Find the batsman participating in 2019 who has the best average of
scoring runs in his career. Solve using Dataframe or datasets.

"""
# as we can use data loading fields as list or we can use direct files
"""
list1 = [(1,"Rohit Sharma","India",200,100.2),
(1,"Virat Kohli","India",100,98.02),
(1,"Steven Smith","Aus",77,79.23),
(35,"Clive Lloyd","WI",29,37.00),
(243,"Rohit Sharma","India",23,150.00),
(243,"Faf du Plesis","SA",17,35.06)]

list2 = [("Rohit_Sharma","India"),
("Steven_Smith","Aus"),
("Virat_Kohli","India")]
"""
fileA = spark.read.format("csv").option("header",True)\
.option("inferSchema", True).option("path","/content/fileA.csv").load()

fileB = spark.read.format("csv").option("header",True)\
.option("inferSchema", True).option("path","/content/fileB.csv").load()

fileA1 = fileA.withColumnRenamed("Batsman", "Batsman_A")
fileB1 = fileB.withColumnRenamed("Batsman", "Batsman_B")
# autoBroadcast Join is turned off
spark.conf.set("spark.sql.autoBroadcastJoinThreshold", -1)
#fileA.show()
#fileB.show()

# the best average of scoring runs in his career
# using broadcast join
from pyspark.sql.functions import broadcast
from pyspark.sql.functions import desc

joining_condition = fileA1.Batsman_A == fileB1.Batsman_B
joinedAvgDf = fileA1.join(broadcast(fileB1), joining_condition, "INNER" )
joinedAvgDf.show()

resultDf = joinedAvgDf.selectExpr("Batsman_A as Batsman",
"ROUND(((sum(RunsScored)over(partition by Batsman_A order by Batsman_A))*1.0/count(Batsman_A)over(partition by Batsman_A order by Batsman_A)), 2) as AvgScore")\
.distinct().orderBy(desc("AvgScore"))

# ----Final Result ---- #

resultDf.show()







+-----------+------------+-----+----------+----------+------------+-----+
|MatchNumber|   Batsman_A| Team|RunsScored|StrikeRate|   Batsman_B| Team|
+-----------+------------+-----+----------+----------+------------+-----+
|          1|Rohit Sharma|India|       200|     100.2|Rohit Sharma|India|
|          1| Virat Kohli|India|       100|     98.02| Virat Kohli|India|
|          1|Steven Smith|  Aus|        77|     79.23|Steven Smith|  Aus|
|        243|Rohit Sharma|India|        23|     150.0|Rohit Sharma|India|
+-----------+------------+-----+----------+----------+------------+-----+

+------------+--------+
|     Batsman|AvgScore|
+------------+--------+
|Rohit Sharma|  111.50|
| Virat Kohli|  100.00|
|Steven Smith|   77.00|
+------------+--------+

